In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
jeopardy = pd.read_csv('jeopardy.csv')
print(jeopardy.head(1))
# print(jeopardy.colu1mns)
jeopardy.columns = jeopardy.columns.str.strip()
print(jeopardy.columns)

   Show Number    Air Date      Round  Category  Value  \
0         4680  2004-12-31  Jeopardy!   HISTORY   $200   

                                            Question      Answer  
0  For the last 8 years of his life, Galileo was ...  Copernicus  
Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')


In [3]:
def norm_qa(string):
    res = string.lower()
    res = re.sub('[,.?!:;\"<>\'\(\)$]','',res)
    return res
jeopardy['clean_question'] = jeopardy['Question'].apply(norm_qa)
jeopardy['clean_answer'] = jeopardy['Answer'].apply(norm_qa)

In [4]:
def norm_dv(string):
    res = re.sub('[,.?!:;\"<>\'\(\)$]','',string) 
    try:
        res = int(res)
    except Exception:
        res = 0
    return res
jeopardy['clean_value'] = jeopardy['Value'].apply(norm_dv)
# print(jeopardy['clean_value'])
jeopardy['Air Date'] = pd.to_datetime(jeopardy['Air Date'])
print(jeopardy['clean_answer'])

0                                               copernicus
1                                               jim thorpe
2                                                  arizona
3                                                mcdonalds
4                                               john adams
5                                                  the ant
6                                           the appian way
7                                           michael jordan
8                                               washington
9                                           crate & barrel
10                                          jackie gleason
11                                                 the cud
12                                     ceylon or sri lanka
13                                               jim brown
14                                            the uv index
15                                                  bulova
16                                             jesse jam

In [5]:
def count_rpt_words(sr):
    split_answer = sr['clean_answer'].split(' ')
    split_question = sr['clean_question'].split(' ')
    match_count = 0
    try:
        split_answer.remove('the')
    except Exception:
        pass
    if len(split_answer) == 0:
        return 0
    for each in split_answer:
        if each in split_question:
            match_count += 1
    return match_count
answer_in_question = jeopardy.apply(count_rpt_words, axis = 1)
np.mean(answer_in_question)

0.15265763288164408

In [6]:
jeopardy = jeopardy.sort_values('Air Date', axis = 0)
question_overlap = list()
terms_used = set()
for tp in jeopardy.iterrows():
    split_question = tp[1]['clean_question'].split(' ')
    split_question = [x for x in split_question if len(x) >= 6]
    match_count = 0
    for x in split_question:
        if x in terms_used:
            match_count += 1
        terms_used.add(x)
    if (len(split_question) > 0):
        match_count /= len(split_question)
    question_overlap.append(match_count)
jeopardy['question_overlap'] = question_overlap
print(np.mean(jeopardy['question_overlap']))   

0.687321950416


In [7]:
# question_overlap is fairly high, studying the question from the
# history helps

In [16]:
def filter_800(sr):
    if sr['clean_value'] > 800:
        return 1
    else:
        return 0

jeopardy['high_value'] = jeopardy.apply(filter_800,axis = 1)

def count_word(word):
    low_count = 0
    high_count = 0
    for tp in jeopardy.iterrows():
        split_question = tp[1]['clean_question'].split(' ')
        if word in split_question:
            if tp[1]['high_value'] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count

observed_expected = []
comparison_terms = list(terms_used)[:5]

for term in comparison_terms:
        observed_expected.append(count_word(term))

print(observed_expected)       

[(1, 4), (1, 1), (0, 1), (0, 1), (0, 2)]


In [32]:
high_value_count = len(jeopardy[jeopardy['high_value'] == 1])
low_value_count = len(jeopardy[jeopardy['high_value'] == 0])
chi_squared = list()
from scipy.stats import chisquare
for ob in observed_expected:
    total = ob[0] + ob[1]
    total_prop = total/len(jeopardy)
    exp_high = total_prop*high_value_count
    exp_low = total_prop*low_value_count
    (chisq, p) = chisquare(ob,f_exp = [exp_high,exp_low])
    chi_squared.append((chisq,p))
print(chi_squared)

1.4335716785839292 3.566428321416071
0.5734286714335717 1.4265713285664283
0.28671433571678584 0.7132856642832142
0.28671433571678584 0.7132856642832142
0.5734286714335717 1.4265713285664283
[(0.18383953104516373, 0.66809416232506025), (0.44487748166127949, 0.50477764875459963), (0.40196284612688399, 0.52607729857054686), (0.40196284612688399, 0.52607729857054686), (0.80392569225376798, 0.36992223780795708)]


In [ ]:
# all the p-values are quite high, meaning there are no statistically
# significant results, the actual distibution between high and low
# are caused by statistical fluctuations